In [1]:
# Import Libraries
import pandas as pd
from pathlib import Path

In [2]:
# Load school data from csv
school_data = Path("Resources/schools_complete.csv")
student_data = Path("Resources/students_complete.csv")

In [3]:
# Create dataframes for school data and student data
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

In [4]:
# Merge school and student dataframes on school name
students_merge_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
students_merge_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
# District Summary

In [6]:
# Calculate the total number of unique schools
school_count = len(students_merge_df["school_name"].unique())
school_count

15

In [7]:
# Calculate the total number of students
student_count = len(students_merge_df['student_name'])
student_count

39170

In [8]:
# Calculate the total budget
total_budget = sum(school_df['budget'])
total_budget

24649428

In [9]:
# Calculate the average (mean) math score
average_math_score = students_merge_df['math_score'].mean()
average_math_score

78.98537145774827

In [10]:
# Calculate the average (mean) reading score
average_reading_score = students_merge_df['reading_score'].mean()
average_reading_score

81.87784018381414

In [11]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = students_merge_df[(students_merge_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [12]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = students_merge_df[(students_merge_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) *100
passing_reading_percentage

85.80546336482001

In [13]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = students_merge_df[
    (students_merge_df["math_score"] >= 70) & (students_merge_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [14]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools":[school_count], 
    "Total Students":[student_count],
    "Total Budget":[total_budget],
    "Average Math Score":[average_math_score],
    "Average Reading Score":[average_reading_score],
    "% Passing Math":[passing_math_percentage],
    "% Passing Reading":[passing_reading_percentage],
    "% Overal Passing":[overall_passing_rate]
    })

    # Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overal Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [15]:
# School Summary

In [35]:
# Use the code provided to select all of the school types
school_types = school_df.set_index(["school_name"])["type"]

In [36]:
# Calculate the total student count per school
per_school_counts = students_merge_df["school_name"].value_counts()
per_school_counts.name = "Total Students"

In [37]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_df.groupby("school_name") ["budget"].sum()
per_school_budget.name = "Total School Budget"
per_school_capita = per_school_budget / per_school_counts
per_school_capita.name = "Per Student Budget"

In [38]:
# Calculate the average test scores per school
per_school_math = students_merge_df.groupby("school_name") ["math_score"].mean()
per_school_math.name = "Average Math Score"
per_school_reading = students_merge_df.groupby("school_name") ["reading_score"].mean()
per_school_reading.name = "Average Reading Score"

In [39]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math =  students_merge_df.loc[students_merge_df["math_score"] >=70]
school_students_passing_math = students_passing_math.groupby("school_name") ["math_score"].count()

In [40]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = students_merge_df.loc[students_merge_df["reading_score"] >=70]
school_students_passing_reading = students_passing_reading.groupby("school_name") ["reading_score"].count()

In [41]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = students_merge_df[
    (students_merge_df["reading_score"] >= 70) & (students_merge_df["math_score"] >= 70)]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [42]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_math.name = "% Passing Math"
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
per_school_passing_reading.name = "% Passing Reading"
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
overall_passing_rate.name = "% Overall Passing"

In [51]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame([school_types,
                                per_school_counts,
                                per_school_budget,
                                per_school_capita,
                                per_school_math,
                                per_school_reading,
                                per_school_passing_math,
                                per_school_passing_reading,
                                overall_passing_rate])

# Schools are column headers and calculations are row indexes, need to transpose
per_school_summary = per_school_summary.transpose()
# Put schools in alphabetical order
per_school_summary = per_school_summary.sort_index()
# Rename type column to Type
per_school_summary.rename(columns = {"type":"Type"}, inplace = True)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.93328,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.15802,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [25]:
# Highest-Performing Schools (by % Overall Passing)